# Exploring the Galaxy Toys Example

We are now going to explore the Galaxy Toys example a bit more. Let's first see how to create and solve the model using Gurobi. Then, how can we get the sensitivity analysis information out?

In [ ]:
# import the two Gurobi parts we need
import gurobipy as gp
from gurobipy import GRB

## Very Simple Approach

We are going to start with the most basic and "simple" way to create and solve our small LP using Gurobi. We will do it steps, with each step in its own code cell. (You can, of course, put all these steps into a single code cell.) NOTE: We should probably put a `try` block around all of this code to catch any runtime errors. 

Recall that our formulation looked like this:

| | | |
| --- | --- | --- |
| Let | | |
| $x_{s}$ | = | number of lots (dozens) of Space Rays to produce next week |
| $x_{p}$ | = | number of lots (dozens) of Phasers to produce next week |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $8x_{s}$ | $+$ | $5x_{p}$ | | | |
| s.t. | $2x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $1200$ | {plastic pounds} |
| | $3x_{s}$ | $+$ | $4x_{p}$ | $\le$ | $2400$ | {minutes of production} |
| | $1x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $800$ | {overall production limit} |
| | $1x_{s}$ | $-$ | $1x_{p}$ | $\le$ | $450$ | {mix of products produced} |
| | $x_{s}$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $x_{p}$ | $\ge$ | $0$ | {non-negativity} |

In [ ]:
# Create the model object
m = gp.Model('galaxy_toys')

In [ ]:
# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

# update the model
m.update()

In [ ]:
# Create decision variables
# We tell the solver that the variables are continuous,
#   their names, and their lower bounds
x_s = m.addVar(vtype=GRB.CONTINUOUS, name='x_s', lb=0.0)
x_p = m.addVar(vtype=GRB.CONTINUOUS, name='x_p', lb=0.0)

# update the model
m.update()

In [ ]:
# Add the objective function
m.setObjective(8 * x_s + 5 * x_p)
m.update()

In [ ]:
# Add the constraints
# We can simply write out the constraints for the first parameter
# The second parameter names the constraint
m.addConstr(2*x_s + x_p <= 1200, name='plastic')
m.addConstr(3*x_s + 4*x_p <= 2400, name='labor')
m.addConstr(x_s + x_p <= 800, name='total_production')
m.addConstr(x_s - x_p <= 450, name='product_mix')
m.update()

In [ ]:
# solve
m.optimize()

### Getting the Results

We now want to get the results and print them in a little nicer format. 

In [ ]:
# Get the results out
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Sensitivity Analysis

We can easily get the reduced cost and the range of optimality for each variable. Similarly, we can extract the shadow price and the range feasibility for each constraint.

In [ ]:
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')

In [ ]:
# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Full Professor of Business Analytics at William \& Mary.**